In [23]:
import time
from ore_algebra import *
DOP, t, D = DifferentialOperators()
# The following may be overridden in input file, but need not be specified there
#
#
defaultPrecision=1e-1000;
# Do not forget to increase the precision of initial conditions if going beyond e-100
#
#
#
precision=[]; #if empty we will use the default precision for all systems
periodsOnly=true; 
paths=[]; #unless overridden we will convert this to a list of straight paths

#load("~/git-projects/periods/integrate/data/current.sage")
load("~/git-projects/period-suite/current.sage")
steps=len(allODEs);
cohomologies=[];
allTransMats=[1..steps];

if len(paths)==0:
    paths=[[0,1] for j in [1..steps]]
    
if len(precision)==0:
    precision=[defaultPrecision for j in [1..steps]]


t0=time.time();


print "Integrating systems", 1, "through", steps;
for j in [1..steps]:
    if periodsOnly and j==steps:
        noOfdeqs=geometricGenus;
    else:
        noOfdeqs=len(allODEs[j-1]);
    deqs=[deq.numerator() for deq in allODEs[j-1][0:noOfdeqs]];
    print "";
    print "Integrating system number", j;
    print "";
    %time transitionMatricies = [deqs[i-1].numerical_transition_matrix(paths[j-1], precision[j-1],assume_analytic=true) for i in [1..noOfdeqs]]
    allTransMats[j-1]=transitionMatricies;
    print "Maximal error in transition matricies:"
    print [max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies]
    print "The largest error:"
    print max([max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies])
    cohomology=[1..noOfdeqs];
    for i in [1..noOfdeqs]:
        if j==1:
            init=Matrix(inits[i-1]);
            a = init.nrows(); b = init.ncols();
            init =  MatrixSpace(ComplexBallField(2000), a, b)(init);
        else:
            init=Matrix(rawInits[j-2][i-1])*cohomologies[j-2];
        transitionMatrix=transitionMatricies[i-1];
        cohomology[i-1]=(transitionMatrix * init).row(0);
    cohomologies=cohomologies+[Matrix(cohomology)];
    
periodsWithError=Matrix(cohomologies[-1].rows()[0:geometricGenus]);
maximalError=max(periodsWithError.apply_map(lambda x : x.diameter()).list());
periods=periodsWithError.apply_map(lambda x : x.mid());
print "\nAccumulated maximal error:", maximalError
t1=time.time();
print "\nIt took", t1-t0, "seconds in total.\n"
print periods


Integrating systems 1 through 1

Integrating system number 1



ZeroDivisionError: rational division by zero

In [11]:
#### Only for K3s
##
## Try a few values for "scale", do not exceed the negative exponent of accumulated error
##
scale = 90
##
## verify that you found honest relations by checking suspected relations using more digits
##
verify= 120
##
##

load("K3_intersection_matrix.sage")

periodVector=periods*intersectionMatrix.inverse()
u1 = periodVector.apply_map(real)[0].list()
u2 = periodVector.apply_map(imag)[0].list()
M = matrix([u1,u2])

proj = M.transpose()
Proj = (10^scale*proj).apply_map(lambda x : 10^3*x.round())
lattice = block_matrix([[Proj, matrix.identity(Proj.dimensions()[0])]])
reducedLattice = lattice.LLL()

Proj2 = (10^verify*proj).apply_map(lambda x : x.round())
testRelations = [v[2:23] * Proj2 for v in reducedLattice.rows()]

norms1 = [reducedLattice.row(i)[2:23].norm().n() for i in [0..20]]
norms2 = [testRelations[i].norm().n()+reducedLattice.row(i)[2:23].norm().n() for i in [0..20]]

norms=norms1

sortedNorms=sorted(norms)
consecutiveRatios = [sortedNorms[i]/sortedNorms[i-1] for i in [1..20]]

j=0; rank=1;
while j <= 19 :
    if consecutiveRatios[j] > 100 :
        j=20
    rank = rank + 1
    j=j+1
    
if sortedNorms[0] > 50 :
    rank = 1

print "\nPicard rank appears to be", rank, "\n"
print "Check to see if this makes sense:\n"

print "01","||", "---hyperplane section---"
j=0
while j <= 7 :
    print "0" + str(j+2), "||", norms[j], testRelations[j]
    j=j+1
while j <= 20 :
    print j+2, "||", norms[j], testRelations[j]
    j=j+1



Picard rank appears to be 18 

Check to see if this makes sense:

01 || ---hyperplane section---
02 || 1.41421356237310 (0, 0)
03 || 1.41421356237310 (0, 0)
04 || 1.41421356237310 (0, 0)
05 || 1.41421356237310 (0, 0)
06 || 1.73205080756888 (0, 1)
07 || 1.73205080756888 (0, 0)
08 || 1.73205080756888 (0, -1)
09 || 1.73205080756888 (0, 0)
10 || 1.73205080756888 (0, 1)
11 || 1.73205080756888 (0, 0)
12 || 2.00000000000000 (0, 0)
13 || 2.00000000000000 (0, 0)
14 || 2.23606797749979 (0, 1)
15 || 2.00000000000000 (0, 0)
16 || 2.00000000000000 (-1, 0)
17 || 1.73205080756888 (0, -1)
18 || 1.73205080756888 (0, 0)
19 || 4.28673296863418e44 (-173719087909469229906316703792442564459910454513491577005064708940513285551, 108517336578602628859792573478231584309402692410319262084681147322820412043)
20 || 8.48887942328466e44 (-813911998441755212594829197316081453500455667288311428485290267067943070614, -351370132025828631443251728205353138346914267564479123577182777888689569173)
21 || 8.55050008890029e4

In [114]:
rel1=reducedLattice[0:19,2:23]
rel1.row_space()

Free module of degree 21 and rank 19 over Integer Ring
Echelon basis matrix:
[ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1]
[ 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1]
[ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -2  0]
[ 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -2 -1]
[ 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0]
[ 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0 -2  0]
[ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0 -2 -1]
[ 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0 -2 -1]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 -5 -3]
[ 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0 -2 -1]
[ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 -2 -1]
[ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0 -2 -1]
[ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1 -1]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0 -2 -2]
[ 0  0  0  0 

In [101]:
rel2=reducedLattice[0:17,2:23]
rel2.row_space()

Free module of degree 21 and rank 17 over Integer Ring
Echelon basis matrix:
[ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0]
[ 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2 -2 -1]
[ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  2 -2 -1]
[ 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  2 -2 -2]
[ 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  1 -2 -1]
[ 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  2 -4 -2]
[ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  2 -4 -3]
[ 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 -1  0  1 -1 -1]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0 -2  0  1 -2 -2]
[ 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1  0  1 -1 -1]
[ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 -2 -1]
[ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0 -2 -1]
[ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  1 -2 -2]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0 -2 -2]
[ 0  0  0  0 

In [88]:
Out[86]==Out[84]

False

In [115]:
block_matrix([[rel1],[rel2]]).row_space()

Free module of degree 21 and rank 19 over Integer Ring
Echelon basis matrix:
[ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1]
[ 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1]
[ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -2  0]
[ 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -2 -1]
[ 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0]
[ 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0 -2  0]
[ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0 -2 -1]
[ 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0 -2 -1]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 -5 -3]
[ 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0 -2 -1]
[ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 -2 -1]
[ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0 -2 -1]
[ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1 -1]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0 -2 -2]
[ 0  0  0  0 

In [14]:
deqs[0].local_basis_monomials(0)

[1, x^2, x^4, x^6]